In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
path = '/content/drive/MyDrive/security/'

In [3]:
!pip install pytorch-pretrained-bert pytorch-nlp

     |████████████████████████████████| 133kB 5.1MB/s 
     |████████████████████████████████| 92kB 5.3MB/s 
     |████████████████████████████████| 133kB 8.7MB/s 
     |████████████████████████████████| 81kB 5.6MB/s 
     |████████████████████████████████| 7.5MB 8.9MB/s 
ERROR: botocore 1.20.69 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [4]:
# Import Libraries

import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import re
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
train=pd.read_csv(path+'train.csv')
test=pd.read_csv(path+'test.csv')
submission=pd.read_csv(path+'sample_submission.csv')

In [6]:
def alpha_num(text):
      return re.sub(r'[^a-zA-z0-9\s]', '', text)

train['full_log'] = train['full_log'].str.lower().apply(alpha_num)
test['full_log'] = test['full_log'].str.lower().apply(alpha_num)   

In [7]:
train['full_log'] = train['full_log'].str.replace(r'[0-9]', '<num>')
test['full_log'] = test['full_log'].str.replace(r'[0-9]', '<num>')

In [8]:
# Create sentence and label lists
sentences = train.full_log.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = train.level.values

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in tqdm(sentences)]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

100%|██████████| 472972/472972 [28:00<00:00, 281.53it/s]

Tokenize the first sentence:
['[CLS]', 'sep', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', 'local', '##hos', '##t', 'ki', '##bana', 'type', '##er', '##ror', '##time', '##sta', '##mp', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', 't', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', 'z', '##tag', '##s', '[', 'warnings', '##tat', '##sco', '##lle', '##ction', ']', 'pi', '##d', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', '<', 'nu', '##m', '>', 'level', '##er', '##ror', '##er', '##ror', '##mes', '##sa', '##gen', '##o', 'living', 'connections', '##name', '##er', '##ror', '##sta', '##cker', '##ror', 'no', 'l

In [10]:
MAX_LEN = 128

In [ ]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [ ]:
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [ ]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [ ]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [ ]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=7)

In [ ]:
# Create sentence and label lists
sentences = test.full_log.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]


tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
 # batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_set = []

for i in range(len(true_labels)):
  matthews = matthews_corrcoef(true_labels[i],
                 np.argmax(predictions[i], axis=1).flatten())
  matthews_set.append(matthews)